In [1]:
import os
import glob
import netCDF4
import logging
import numpy as np
import cmocean
import emodnetchemistry
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
import calendar
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
plt.rcParams.update({'font.size': 16})
plt.rc('figure', facecolor='w')

In [21]:
figdir = "../figures/WOA2018/"
woadataurl = "https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/oxygen/all/1.00/woa18_all_o00_01.nc"
divafile = "/data/EMODnet/Eutrophication/Products/Results-0.1/Water body dissolved oxygen concentration_background_test_it2000_800km.nc"
deltalon = 0.1
deltalat = 0.1
domain = [-45., 70., 24., 83.]
longrid = np.arange(domain[0], domain[1] + .0001, deltalon)
latgrid = np.arange(domain[2], domain[3] + .0001, deltalat)
if not os.path.exists(figdir):
    os.makedirs(figdir)

In [3]:
def read_oxy_woa(datafile, domain, filetype="woa"):
    with netCDF4.Dataset(datafile, "r") as nc:
        lon = nc.variables["lon"][:]
        lat = nc.variables["lat"][:]
        goodlon = np.where( (lon >= domain[0]) & (lon <= domain[1]))[0]
        goodlat = np.where( (lat >= domain[2]) & (lat <= domain[3]))[0]
        lon = lon[goodlon]
        lat = lat[goodlat]
        depth = nc.variables["depth"][:]
        
        if filetype == "woa":
            oxy = nc.variables["o_an"][0,:,goodlat,goodlon]
        elif filetype == "diva":
            oxy = nc.variables["Water body dissolved oxygen concentration"][:,goodlat,goodlon]
        
    return lon, lat, depth, oxy

In [4]:
def plot_oxy_map(llon, llat, field, depth, figname=None):
    fig = plt.figure(figsize=(10, 10))
    ax = plt.subplot(111)
    pcm = m.pcolormesh(llon, llat, field, latlon=True, vmin=200., vmax=375.)
    cb = plt.colorbar(pcm, shrink=.6, extend="both")
    cb.set_label(r"$\mu$moles/kg", rotation=0, ha="left")
    cb.set_ticks(np.arange(200., 376., 25))
    m.drawmeridians(np.arange(-40., 60., 20.), zorder=2, labels=[0, 0, 0, 1], fontsize=14,  
                    linewidth=.25)
    m.drawparallels(np.arange(20., 80., 10.), zorder=2, labels=[1, 0, 0, 0], fontsize=14, 
                    linewidth=.25)
    m.fillcontinents(color=".85", zorder=3)
    m.drawcoastlines(linewidth=0.1, zorder=4)
    plt.title("Oxygen concentration at {} m".format(depth))
    if figname is not None:
        plt.savefig(figname, dpi=300, bbox_inches="tight")
    # plt.show()
    plt.close()

In [24]:
m = Basemap(projection='merc', llcrnrlon=domain[0], llcrnrlat=domain[2],
            urcrnrlon=domain[1], urcrnrlat=domain[3],
            lat_ts=0.5 * (domain[2] + domain[3]), resolution='i')

In [25]:
lon1, lat1, depth1, oxy1 = read_oxy_woa(woadataurl, domain, filetype='woa')
lon2, lat2, depth2, oxy2 = read_oxy_woa(divafile, domain, filetype='diva')
llon1, llat1 = np.meshgrid(lon1, lat1)
llon2, llat2 = np.meshgrid(lon2, lat2)
depth2plot = 0

## Loop on the depths

In [37]:
def plot_WOA_DIVAnd_comparison(lon1, lat1, field1, lon2, lat2, field2, depth, figname=None,
                              vmin=200., vmax=375.):

    fig = plt.figure(figsize=(12, 10))
    ax1 = plt.subplot(121)
    pcm = m.pcolormesh(lon1, lat1, field1, latlon=True, vmin=vmin, vmax=vmax)
    m.drawmeridians(np.arange(-40., 70., 20.), zorder=2, labels=[0, 0, 0, 1], fontsize=14,  
                    linewidth=.25)
    m.drawparallels(np.arange(20., 81., 10.), zorder=2, labels=[1, 0, 0, 0], fontsize=14, 
                    linewidth=.25)
    m.fillcontinents(color=".85", zorder=3)
    m.drawcoastlines(linewidth=0.1, zorder=4)
    plt.title("World Ocean Atlas 2018 at {} m".format(int(depth)))

    ax2 = plt.subplot(122)
    pcm = m.pcolormesh(lon2, lat2, field2, latlon=True, vmin=vmin, vmax=vmax)
    m.drawmeridians(np.arange(-40., 70., 20.), zorder=2, labels=[0, 0, 0, 1], fontsize=14,  
                    linewidth=.25)
    m.drawparallels(np.arange(20., 81., 10.), zorder=2, labels=[1, 0, 0, 0], fontsize=14, 
                    linewidth=.25)
    m.fillcontinents(color=".85", zorder=3)
    m.drawcoastlines(linewidth=0.1, zorder=4)
    plt.title("DIVAnd")


    cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.05])
    cb = plt.colorbar(pcm,  cax=cbar_ax, extend="both", orientation="horizontal")
    cb.set_label(r"$\mu$moles/kg", rotation=0, ha="left")
    cb.set_ticks(np.arange(200., 376., 25))
    plt.savefig(figname, dpi=300, bbox_inches="tight")
    plt.close()

In [42]:
for index2, d2 in enumerate(depth2):
    print("Working on depth {}".format(d2))
    index1 = np.where(depth1 == d2)[0][0]

    
    figname = os.path.join(figdir, "oxygen_DIVAnd_WOA_year_{}.jpg".format(str(int(d2)).zfill(4)))
    plot_WOA_DIVAnd_comparison(llon1, llat1, oxy1[index1,:,:], 
                               llon2, llat2, oxy2[index2,:,:], 
                               d2, figname=figname)

Working on depth 0.0
Working on depth 10.0
Working on depth 20.0
Working on depth 30.0
Working on depth 50.0
Working on depth 75.0
Working on depth 100.0
Working on depth 125.0
Working on depth 150.0
Working on depth 200.0
Working on depth 250.0
Working on depth 300.0
Working on depth 400.0
Working on depth 500.0
Working on depth 600.0
Working on depth 700.0
Working on depth 800.0
Working on depth 900.0
Working on depth 1000.0
Working on depth 1100.0
Working on depth 1200.0
Working on depth 1300.0
Working on depth 1400.0
Working on depth 1500.0
Working on depth 1750.0
Working on depth 2000.0
Working on depth 2500.0
Working on depth 3000.0
Working on depth 3500.0
Working on depth 4000.0
Working on depth 4500.0
Working on depth 5000.0
Working on depth 5500.0


In [38]:
depth = 0.0
figname = os.path.join(figdir, "oxygen_DIVAnd_WOA_year_{}.jpg".format(str(int(depth)).zfill(4)))
plot_WOA_DIVAnd_comparison(llon1, llat1, oxy1[0,:,:], llon2, llat2, oxy2[0,:,:], depth, figname=figname)